In [ ]:
from nbdev import *


In [ ]:
# | default_exp core


# Core module

> API details.

In [ ]:
# |hide
from nbdev.showdoc import *


In [ ]:
# |export
from scip_workflows.common import *


In [ ]:
# |export
import matplotlib.gridspec as gridspec
import yaml
import zarr
from aicsimageio import AICSImage
from matplotlib.colors import LinearSegmentedColormap, Normalize
import math
from scip.masking import remove_regions_touching_border


# Plot manual gate images

In [ ]:
# |export
def plot_gate_zarr(
    sel,
    df,
    mask,
    maxn=200,
    sort=None,
    channels=[0],
    bbox=True,
    ncols=5,
    cmaps=None,
    qq=(0, 1),
    extent=None
):
    
    compute_extent = extent is None
    
    df = df.loc[sel]

    if len(df) > maxn:
        df = df.sample(n=maxn)

    if sort is not None:
        df = df.sort_values(by=sort)

    i = 0
    pixels = [None] * len(df)
    
    if compute_extent:
        extent = numpy.full(
            (df.shape[0], 2, len(channels)), dtype=float, fill_value=numpy.nan
        )
    for path, gdf in df.groupby("meta_path"):
        z = zarr.open(path, mode="r")
        for (idx, r) in gdf.iterrows():
            pixels_ = z[r["meta_zarr_idx"]]
            pixels_ = pixels_.reshape(z.attrs["shape"][r["meta_zarr_idx"]])[channels]

            if bbox:
                minr, minc, maxr, maxc = (
                    int(r[f"meta_{mask}_bbox_minr"]),
                    int(r[f"meta_{mask}_bbox_minc"]),
                    int(r[f"meta_{mask}_bbox_maxr"]),
                    int(r[f"meta_{mask}_bbox_maxc"]),
                )
                pixels_ = pixels_[:, minr:maxr, minc:maxc]

            if compute_extent:
                extent[i, 0] = numpy.quantile(
                    pixels_.reshape(pixels_.shape[0], -1), q=qq[0], axis=1
                )
                extent[i, 1] = numpy.quantile(
                    pixels_.reshape(pixels_.shape[0], -1), q=qq[1], axis=1
                )

            pixels[df.index.get_loc(idx)] = pixels_

            i += 1

    if compute_extent:
        min_ = extent[:, 0].min(axis=0)
        max_ = extent[:, 1].max(axis=0)
    else:
        min_ = extent[:, 0]
        max_ = extent[:, 1]

    ncols = min(df.shape[0], ncols)
    nrows = int(math.ceil(len(df) / ncols))
    fig, axes = plt.subplots(
        ncols=ncols,
        nrows=nrows,
        dpi=50,
        figsize=(ncols * 2 * len(channels), nrows * 2),
        squeeze=False,
    )
    axes = axes.ravel()

    if cmaps == None:
        cmaps = [plt.get_cmap("viridis")] * len(channels)

    for i, pixels_ in enumerate(pixels):
        tmp = (pixels_ - min_[:, numpy.newaxis, numpy.newaxis]) / (max_ - min_)[
            :, numpy.newaxis, numpy.newaxis
        ]
        pixels[i] = numpy.hstack([cm(p) for cm, p in zip(cmaps, tmp)])

    for i, (ax, pixels_) in enumerate(zip(axes, pixels)):
        ax.imshow(pixels_)
    for ax in axes:
        ax.set_axis_off()


In [ ]:
import pickle

with open(
    "/home/maximl/scratch/data/vsc/datasets/weizmann/EhV/v2/scip/011020222241/adata.pickle",
    "rb",
) as fh:
    adata = pickle.load(fh)


In [ ]:
indices = [
    adata[adata.obs.meta_label == "mcp-_psba+"].obs.iloc[0].name,
    adata[adata.obs.meta_label == "mcp-_psba+"].obs.iloc[11].name,
    adata[adata.obs.meta_label == "mcp+_psba+"].obs.iloc[10].name,
    adata[adata.obs.meta_label == "mcp+_psba+"].obs.iloc[100].name,
    adata[adata.obs.meta_label == "mcp+_psba-"].obs.iloc[499].name,
    adata[adata.obs.meta_label == "mcp+_psba-"].obs.iloc[3].name,
    adata[adata.obs.meta_label == "mcp-_psba-"].obs.iloc[42].name,
    adata[adata.obs.meta_label == "mcp-_psba-"].obs.iloc[89].name,
]


In [ ]:
N = 255
cmaps = [
    LinearSegmentedColormap.from_list(
        "test", [(0, 0, 0, 1), (227 / N, 161 / N, 220 / N, 1)]
    ),  # dapi
    LinearSegmentedColormap.from_list(
        "test", [(0, 0, 0, 1), (228 / N, 204 / N, 7 / N, 1)]
    ),  # mcp
    LinearSegmentedColormap.from_list(
        "test", [(0, 0, 0, 1), (220 / N, 41 / N, 41 / N, 1)]
    ),  # psba
    plt.get_cmap("Greys"),
    plt.get_cmap("Greys"),
    plt.get_cmap("viridis"),
]


In [ ]:
tmp = plot_gate_zarr(
    [True] * len(indices),
    adata.obs.loc[indices],
    "li",
    channels=[4, 1, 6, 0, 5, 3],
    ncols=2,
    cmaps=cmaps,
    qq=[0.05, 0.97],
)


In [1]:
# |export
def plot_gate_zarr_channels(
    selectors,
    df,
    mask,
    maxn=20,
    sort=None,
    show_mask=False,
    main_channel=3,
    smooth=0.75,
    channel_ind=[0],
    channel_names=["c"],
    extent=None
):
    
    compute_extent = extent is None

    dfs = []
    for i, sel in enumerate(selectors):
        tmp_df = df[sel].copy()

        if len(tmp_df) > maxn:
            tmp_df = tmp_df.sample(n=maxn)

        if sort is not None:
            tmp_df = tmp_df.sort_values(by=sort)

        tmp_df["sel"] = i
        dfs.append(tmp_df)
    df = pandas.concat(dfs)

    nchannels = len(channel_ind)

    images = {}
    masks = {}
    
    if compute_extent:
        extent = numpy.empty(shape=(nchannels, 2), dtype=float)
        extent[:, 0] = numpy.inf
        extent[:, 1] = -numpy.inf

    for path, gdf in df.groupby("meta_path"):
        z = zarr.open(path, mode="r")
        for (idx, r) in gdf.iterrows():
            pixels = z[r["meta_zarr_idx"]]
            pixels = pixels.reshape(z.attrs["shape"][r["meta_zarr_idx"]])[channel_ind]

            minr, minc, maxr, maxc = (
                int(r[f"meta_{mask}_bbox_minr"]),
                int(r[f"meta_{mask}_bbox_minc"]),
                int(r[f"meta_{mask}_bbox_maxr"]),
                int(r[f"meta_{mask}_bbox_maxc"]),
            )

            images[r["sel"]] = images.get(r["sel"], []) + [
                pixels[:, minr:maxr, minc:maxc]
            ]
            if show_mask:
                m = li.get_mask(
                    dict(pixels=pixels), main_channel=main_channel, smooth=smooth
                )
                m = remove_regions_touching_border(m, bbox_channel_index=main_channel)
                arr = m["mask"][:, minr:maxr, minc:maxc]
            else:
                arr = numpy.full(shape=pixels.shape, dtype=bool, fill_value=True)[
                    :, minr:maxr, minc:maxc
                ]
            masks[r["sel"]] = masks.get(r["sel"], []) + [
                numpy.where(arr, numpy.nan, arr)
            ]

            p = numpy.where(arr, pixels[:, minr:maxr, minc:maxc], numpy.nan)
            
            if compute_extent:
                extent[:, 0] = numpy.nanmin(
                    numpy.array(
                        [extent[:, 0], numpy.nanmin(p.reshape(nchannels, -1), axis=1)]
                    ),
                    axis=0,
                )
                extent[:, 1] = numpy.nanmax(
                    numpy.array(
                        [extent[:, 1], numpy.nanmax(p.reshape(nchannels, -1), axis=1)]
                    ),
                    axis=0,
                )

    fig = plt.figure(dpi=75, figsize=(len(channel_ind) * 1, len(df) * 0.8))
    grid = gridspec.GridSpec(1, len(selectors), figure=fig, wspace=0.1)
    cmap = plt.get_cmap("viridis")
    norms = [Normalize(vmin=a, vmax=b) for a, b in extent]

    gs = {k: grid[0, k].subgridspec(len(v), nchannels) for k, v in images.items()}
    for k, v in images.items():
        for i, image in enumerate(v):
            for j, (p, m, norm) in enumerate(zip(image, masks[k][i], norms)):
                ax = plt.Subplot(fig, gs[k][i, j])
                ax.imshow(cmap(norm(p)))
                if show_mask:
                    ax.imshow(m, alpha=0.3, cmap="Blues")
                ax.set_axis_off()
                fig.add_subplot(ax)
                if i == 0:
                    ax.set_title(channel_names[j])


In [ ]:
# |export
def plot_gate_czi(
    sel, df, maxn=200, sort=None, channels=[0], masks_path_col=None, extent=None
):
    df = df.loc[sel]

    if len(df) > maxn:
        df = df.sample(n=maxn)

    if sort is not None:
        df = df.sort_values(by=sort)

    compute_extent = False
    if extent is None:
        compute_extent = True
        extent = numpy.full(
            (df.shape[0], 2, len(channels)), dtype=float, fill_value=numpy.nan
        )

    pixels = []
    masks = []
    ids = []
    i = 0
    for path, gdf in df.groupby(["meta_path"]):
        ai = AICSImage(path, reconstruct_mosaic=False)
        for scene, gdf2 in gdf.groupby(["meta_scene"]):
            ai.set_scene(scene)
            for tile, gdf3 in gdf2.groupby(["meta_tile"]):
                print(tile, scene, end=" ")
                for (idx, r) in gdf3.iterrows():
                    pixels_ = ai.get_image_data("CXY", Z=0, T=0, C=channels, M=tile)
                    minr, minc, maxr, maxc = (
                        int(r["meta_bbox_minr"]),
                        int(r["meta_bbox_minc"]),
                        int(r["meta_bbox_maxr"]),
                        int(r["meta_bbox_maxc"]),
                    )

                    if compute_extent:
                        extent[i, 0] = (
                            pixels_[:, minr:maxr, minc:maxc]
                            .reshape(pixels_.shape[0], -1)
                            .min(axis=1)
                        )
                        extent[i, 1] = (
                            pixels_[:, minr:maxr, minc:maxc]
                            .reshape(pixels_.shape[0], -1)
                            .max(axis=1)
                        )
                    pixels.append(pixels_[:, minr:maxr, minc:maxc])

                    if "meta_id" in r:
                        ids.append(r.meta_id)
                    else:
                        ids.append(idx[-1])

                    if masks_path_col is not None:
                        mask = numpy.load(r[masks_path_col])[:, minr:maxr, minc:maxc]
                        masks.append(mask)

                    i += 1

    if compute_extent:
        min_ = extent[:, 0].min(axis=0)
        max_ = extent[:, 1].max(axis=0)
    else:
        min_ = extent[:, 0]
        max_ = extent[:, 1]

    ncols = min(df.shape[0], 5)
    nrows = int(math.ceil(len(df) / ncols))
    fig, axes = plt.subplots(
        ncols=ncols, nrows=nrows, dpi=50, figsize=(ncols * 2 * len(channels), nrows * 2)
    )
    axes = axes.ravel()

    for i, (ax, pixels_, id_) in enumerate(zip(axes, pixels, ids)):
        ax.imshow(
            numpy.hstack(
                (pixels_ - min_[:, numpy.newaxis, numpy.newaxis])
                / (max_ - min_)[:, numpy.newaxis, numpy.newaxis]
            )
        )
        if len(masks) > 0:
            ax.imshow(
                numpy.hstack(numpy.where(masks[i] == id_, numpy.nan, 1)),
                cmap="Blues",
                alpha=0.3,
            )

    for ax in axes:
        ax.set_axis_off()


In [ ]:
import pickle

with open(
    "/home/maximl/scratch/data/vsc/datasets/cd7/800/scip/061020221736/adata.pickle",
    "rb",
) as fh:
    adata = pickle.load(fh)


In [ ]:
adata.obs["meta_masks"] = adata.obs[["meta_scene", "meta_tile"]].apply(
    lambda r: "/home/maximl/scratch/data/vsc/datasets/cd7/800/scip/061020221736/masks/%s_%s.npy"
    % (r.meta_scene, r.meta_tile),
    axis=1,
)


In [ ]:
n = 20
plot_gate_czi(
    [True] * n,
    adata.obs.iloc[:n],
    maxn=n,
    channels=[0, 1, 2, 3, 4],
    masks_path_col="meta_masks",
)


In [ ]:
quantiles = adata.to_df().filter(regex="feat_combined_sum").quantile([0.05, 0.95])


In [ ]:
quantiles.loc[
    :, ["feat_combined_sum_%s" % s for s in ["DAPI", "EGFP", "RPe", "APC", "Bright"]]
].T.values


In [ ]:
plot_gate_czi(
    [True] * n,
    adata.obs.iloc[:n],
    maxn=n,
    channels=[0, 1, 2, 3, 4],
    masks_path_col="meta_masks",
    extent=quantiles.loc[
        :,
        ["feat_combined_sum_%s" % s for s in ["DAPI", "EGFP", "RPe", "APC", "Bright"]],
    ].T.values,
)
